In [3]:
import numpy as np
from sklearn.model_selection import GridSearchCV

from keras.layers import Dense, Flatten, Dropout
from keras.models import Sequential
from tensorflow.keras.layers import LayerNormalization
from keras.wrappers.scikit_learn import KerasClassifier

# Assuming X_train, y_train, X_val, y_val, X_test, y_test are defined

def layernorm_create_model(num_layers=1, dropr=0.1, learning_rate=0.001):
    model = Sequential()
    model.add(Flatten())

    for _ in range(num_layers):
        model.add(LayerNormalization())
        model.add(Dense(512, activation='relu', kernel_initializer='lecun_normal', bias_initializer='zeros'))
        model.add(Dropout(dropr))

    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

# Convert Keras model to scikit-learn compatible model
layernorm_classifier = KerasClassifier(build_fn=layernorm_create_model, epochs=10, batch_size=128, verbose=0)

# Define hyperparameters for tuning
param_grid = {
    'num_layers': [1, 2, 3],
    'dropr': [0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30]
}

# Create GridSearchCV
grid = GridSearchCV(estimator=layernorm_classifier, param_grid=param_grid, scoring='accuracy', cv=3)
grid_result = grid.fit(X_train, y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Get the best model
best_model = grid_result.best_estimator_.model

# Train and evaluate the best model
best_model.fit(X_train, y_train, epochs=best_model.epochs, batch_size=best_model.batch_size, verbose=1, validation_data=(X_val, y_val))
best_model_score = best_model.evaluate(X_test, y_test, verbose=0)
print('Best Model Test loss:', best_model_score[0])
print('Best Model Test accuracy:', best_model_score[1])

ModuleNotFoundError: No module named 'keras.wrappers'